# Geogebra look-a-like
Here I have created a geogebra lookalike for Seamus!

In [13]:
from IPython.display import HTML


html_content="""<div class="info-panel">
        <h2>Triangle Angle Sum Demonstration</h2>
        <p>Drag any vertex of the triangle to change its shape. Notice that the sum of the internal angles always remains 180°.</p>
    </div>
    
    <div id="canvas-container">
        <canvas id="grid-canvas" width="600" height="400"></canvas>
        <canvas id="triangle-canvas" width="600" height="400"></canvas>
    </div>
    
    <div class="angle-display">
        <div>Angle A: <span id="angleA">0°</span></div>
        <div>Angle B: <span id="angleB">0°</span></div>
        <div>Angle C: <span id="angleC">0°</span></div>
    </div>
    
    <div class="measurement">Sum of angles: <span id="sumAngles">180°</span></div>
    <div class="measurement">Area of triangle: <span id="triangleArea">0</span> square units</div>
    
    <script>
        // Canvas setup
        const gridCanvas = document.getElementById('grid-canvas');
        const triangleCanvas = document.getElementById('triangle-canvas');
        const gridCtx = gridCanvas.getContext('2d');
        const ctx = triangleCanvas.getContext('2d');
        
        // Coordinate system parameters
        const origin = { x: 300, y: 200 }; // Center of canvas
        const scale = 40; // Pixels per unit
        
        // Triangle vertices (in our coordinate system)
        const vertices = [
            { x: -3, y: -2, color: '#FF4040', name: 'A' }, // Red
            { x: 3, y: -2, color: '#40A040', name: 'B' },  // Green
            { x: 0, y: 3, color: '#4040FF', name: 'C' }    // Blue
        ];
        
        // Selected vertex for dragging
        let selectedVertex = null;
        
        // Convert coordinate system to canvas coordinates
        function toCanvasCoords(point) {
            return {
                x: origin.x + point.x * scale,
                y: origin.y - point.y * scale
            };
        }
        
        // Convert canvas coordinates to our coordinate system
        function fromCanvasCoords(canvasPoint) {
            return {
                x: (canvasPoint.x - origin.x) / scale,
                y: (origin.y - canvasPoint.y) / scale
            };
        }
        
        // Calculate distance between two points
        function distance(p1, p2) {
            return Math.sqrt((p2.x - p1.x) ** 2 + (p2.y - p1.y) ** 2);
        }
        
        // Calculate angle between three points (in degrees)
        function calculateAngle(pointA, pointVertex, pointB) {
            const vectorA = { x: pointA.x - pointVertex.x, y: pointA.y - pointVertex.y };
            const vectorB = { x: pointB.x - pointVertex.x, y: pointB.y - pointVertex.y };
            
            // Calculate dot product
            const dotProduct = vectorA.x * vectorB.x + vectorA.y * vectorB.y;
            
            // Calculate magnitudes
            const magnitudeA = Math.sqrt(vectorA.x ** 2 + vectorA.y ** 2);
            const magnitudeB = Math.sqrt(vectorB.x ** 2 + vectorB.y ** 2);
            
            // Calculate the angle in radians and convert to degrees
            let angleRadians = Math.acos(dotProduct / (magnitudeA * magnitudeB));
            return angleRadians * 180 / Math.PI;
        }
        
        // Calculate triangle area using cross product
        function calculateArea(p1, p2, p3) {
            return 0.5 * Math.abs(
                p1.x * (p2.y - p3.y) + 
                p2.x * (p3.y - p1.y) + 
                p3.x * (p1.y - p2.y)
            );
        }
        
        // Draw the coordinate grid
        function drawGrid() {
            gridCtx.clearRect(0, 0, gridCanvas.width, gridCanvas.height);
            gridCtx.strokeStyle = '#ddd';
            gridCtx.lineWidth = 1;
            
            // Draw grid lines
            for (let x = -7; x <= 7; x++) {
                gridCtx.beginPath();
                const startPoint = toCanvasCoords({ x: x, y: -5 });
                const endPoint = toCanvasCoords({ x: x, y: 5 });
                gridCtx.moveTo(startPoint.x, startPoint.y);
                gridCtx.lineTo(endPoint.x, endPoint.y);
                gridCtx.stroke();
                
                // Add x-axis labels
                if (x !== 0) {
                    const labelPos = toCanvasCoords({ x: x, y: 0.2 });
                    gridCtx.fillStyle = '#888';
                    gridCtx.font = '10px Arial';
                    gridCtx.textAlign = 'center';
                    gridCtx.fillText(x.toString(), labelPos.x, labelPos.y);
                }
            }
            
            for (let y = -5; y <= 5; y++) {
                gridCtx.beginPath();
                const startPoint = toCanvasCoords({ x: -7, y: y });
                const endPoint = toCanvasCoords({ x: 7, y: y });
                gridCtx.moveTo(startPoint.x, startPoint.y);
                gridCtx.lineTo(endPoint.x, endPoint.y);
                gridCtx.stroke();
                
                // Add y-axis labels
                if (y !== 0) {
                    const labelPos = toCanvasCoords({ x: -0.2, y: y });
                    gridCtx.fillStyle = '#888';
                    gridCtx.font = '10px Arial';
                    gridCtx.textAlign = 'right';
                    gridCtx.fillText(y.toString(), labelPos.x, labelPos.y);
                }
            }
            
            // Draw x and y axes (thicker)
            gridCtx.strokeStyle = '#888';
            gridCtx.lineWidth = 2;
            
            // x-axis
            gridCtx.beginPath();
            gridCtx.moveTo(toCanvasCoords({ x: -7, y: 0 }).x, toCanvasCoords({ x: -7, y: 0 }).y);
            gridCtx.lineTo(toCanvasCoords({ x: 7, y: 0 }).x, toCanvasCoords({ x: 7, y: 0 }).y);
            gridCtx.stroke();
            
            // y-axis
            gridCtx.beginPath();
            gridCtx.moveTo(toCanvasCoords({ x: 0, y: -5 }).x, toCanvasCoords({ x: 0, y: -5 }).y);
            gridCtx.lineTo(toCanvasCoords({ x: 0, y: 5 }).x, toCanvasCoords({ x: 0, y: 5 }).y);
            gridCtx.stroke();
            
            // Add origin label
            gridCtx.fillStyle = '#888';
            gridCtx.font = '10px Arial';
            gridCtx.textAlign = 'right';
            gridCtx.fillText('0', toCanvasCoords({ x: -0.1, y: -0.1 }).x, toCanvasCoords({ x: -0.1, y: -0.1 }).y);
        }
        
        // Draw the triangle and all associated information
        function drawTriangle() {
            ctx.clearRect(0, 0, triangleCanvas.width, triangleCanvas.height);
            
            // Draw the triangle sides first
            ctx.beginPath();
            const pA = toCanvasCoords(vertices[0]);
            const pB = toCanvasCoords(vertices[1]);
            const pC = toCanvasCoords(vertices[2]);
            
            ctx.moveTo(pA.x, pA.y);
            ctx.lineTo(pB.x, pB.y);
            ctx.lineTo(pC.x, pC.y);
            ctx.closePath();
            
            ctx.fillStyle = 'rgba(200, 200, 255, 0.3)';
            ctx.fill();
            ctx.strokeStyle = '#333';
            ctx.lineWidth = 2;
            ctx.stroke();
            
            // Draw and label the angles
            const angleColors = [
                'rgba(255, 100, 100, 0.5)', // A - red
                'rgba(100, 255, 100, 0.5)', // B - green
                'rgba(100, 100, 255, 0.5)'  // C - blue
            ];
            
            // Calculate angles
            const angleA = calculateAngle(vertices[1], vertices[0], vertices[2]);
            const angleB = calculateAngle(vertices[2], vertices[1], vertices[0]);
            const angleC = calculateAngle(vertices[0], vertices[2], vertices[1]);
            
            // Draw angle arcs
            drawAngle(pB, pA, pC, angleColors[0], 25);
            drawAngle(pC, pB, pA, angleColors[1], 25);
            drawAngle(pA, pC, pB, angleColors[2], 25);
            
            // Draw side lengths
            const sideAB = distance(vertices[0], vertices[1]).toFixed(2);
            const sideBC = distance(vertices[1], vertices[2]).toFixed(2);
            const sideCA = distance(vertices[2], vertices[0]).toFixed(2);
            
            // Calculate midpoints for labels
            const midAB = { 
                x: (pA.x + pB.x) / 2, 
                y: (pA.y + pB.y) / 2
            };
            const midBC = { 
                x: (pB.x + pC.x) / 2, 
                y: (pB.y + pC.y) / 2
            };
            const midCA = { 
                x: (pC.x + pA.x) / 2, 
                y: (pC.y + pA.y) / 2
            };
            
            // Draw side length labels
            ctx.font = '12px Arial';
            ctx.fillStyle = '#333';
            ctx.textAlign = 'center';
            ctx.fillText(`c = ${sideAB}`, midAB.x, midAB.y + 15);
            ctx.fillText(`a = ${sideBC}`, midBC.x, midBC.y - 5);
            ctx.fillText(`b = ${sideCA}`, midCA.x - 15, midCA.y);
            
            // Draw vertices with labels and coordinates
            for (let i = 0; i < vertices.length; i++) {
                const p = toCanvasCoords(vertices[i]);
                
                // Draw point
                ctx.beginPath();
                ctx.arc(p.x, p.y, 8, 0, Math.PI * 2);
                ctx.fillStyle = vertices[i].color;
                ctx.fill();
                ctx.strokeStyle = '#333';
                ctx.lineWidth = 1;
                ctx.stroke();
                
                // Draw vertex label
                ctx.font = 'bold 14px Arial';
                ctx.fillStyle = '#000';
                ctx.textAlign = 'center';
                ctx.textBaseline = 'middle';
                ctx.fillText(vertices[i].name, p.x, p.y);
                
                // Draw coordinates
                ctx.font = '12px Arial';
                const coordStr = `(${vertices[i].x.toFixed(2)}, ${vertices[i].y.toFixed(2)})`;
                const offsetY = (i === 2) ? -15 : 20; // adjust for different points
                ctx.fillText(coordStr, p.x, p.y + offsetY);
            }
            
            // Update angles and calculations in the HTML
            document.getElementById('angleA').textContent = angleA.toFixed(1) + '°';
            document.getElementById('angleB').textContent = angleB.toFixed(1) + '°';
            document.getElementById('angleC').textContent = angleC.toFixed(1) + '°';
            
            const angleSum = (angleA + angleB + angleC).toFixed(1);
            document.getElementById('sumAngles').textContent = angleSum + '°';
            
            // Calculate and display area
            const area = calculateArea(vertices[0], vertices[1], vertices[2]);
            document.getElementById('triangleArea').textContent = area.toFixed(2);
        }
        
        // Helper function to draw angle arcs
        function drawAngle(p1, vertex, p2, color, radius) {
            // Get angle
            const dx1 = p1.x - vertex.x;
            const dy1 = p1.y - vertex.y;
            const dx2 = p2.x - vertex.x;
            const dy2 = p2.y - vertex.y;
            
            const angle1 = Math.atan2(dy1, dx1);
            const angle2 = Math.atan2(dy2, dx2);
            
            let startAngle = angle1;
            let endAngle = angle2;
            
            // Ensure we're drawing the interior angle
            if (Math.abs(endAngle - startAngle) > Math.PI) {
                if (startAngle > endAngle) {
                    endAngle += Math.PI * 2;
                } else {
                    startAngle += Math.PI * 2;
                }
            }
            
            // Draw angle arc
            ctx.beginPath();
            ctx.moveTo(vertex.x, vertex.y);
            ctx.arc(vertex.x, vertex.y, radius, startAngle, endAngle);
            ctx.closePath();
            ctx.fillStyle = color;
            ctx.fill();
            ctx.strokeStyle = 'rgba(0,0,0,0.3)';
            ctx.stroke();
        }
        
        // Initialize and draw
        function init() {
            drawGrid();
            drawTriangle();
            
            // Add event listeners for dragging
            triangleCanvas.addEventListener('mousedown', onMouseDown);
            triangleCanvas.addEventListener('mousemove', onMouseMove);
            triangleCanvas.addEventListener('mouseup', onMouseUp);
            triangleCanvas.addEventListener('mouseleave', onMouseUp);
            triangleCanvas.addEventListener('touchstart', onTouchStart, { passive: false });
            triangleCanvas.addEventListener('touchmove', onTouchMove, { passive: false });
            triangleCanvas.addEventListener('touchend', onTouchEnd);
        }
        
        // Mouse event handlers
        function onMouseDown(e) {
            const mousePos = getMousePos(triangleCanvas, e);
            const coordPos = fromCanvasCoords(mousePos);
            
            // Check if we clicked on a vertex
            for (let i = 0; i < vertices.length; i++) {
                const vertexCanvasPos = toCanvasCoords(vertices[i]);
                if (distance(mousePos, vertexCanvasPos) < 15) {
                    selectedVertex = i;
                    triangleCanvas.style.cursor = 'grabbing';
                    break;
                }
            }
        }
        
        function onMouseMove(e) {
            const mousePos = getMousePos(triangleCanvas, e);
            const coordPos = fromCanvasCoords(mousePos);
            
            // Change cursor to pointer when over a vertex
            let overVertex = false;
            for (let i = 0; i < vertices.length; i++) {
                const vertexCanvasPos = toCanvasCoords(vertices[i]);
                if (distance(mousePos, vertexCanvasPos) < 15) {
                    triangleCanvas.style.cursor = 'grab';
                    overVertex = true;
                    break;
                }
            }
            
            if (!overVertex && selectedVertex === null) {
                triangleCanvas.style.cursor = 'default';
            }
            
            // If we're dragging a vertex, update its position
            if (selectedVertex !== null) {
                vertices[selectedVertex].x = coordPos.x;
                vertices[selectedVertex].y = coordPos.y;
                drawTriangle();
            }
        }
        
        function onMouseUp(e) {
            selectedVertex = null;
            triangleCanvas.style.cursor = 'default';
        }
        
        // Touch event handlers
        function onTouchStart(e) {
            e.preventDefault();
            if (e.touches.length === 1) {
                const touch = e.touches[0];
                const touchPos = getTouchPos(triangleCanvas, touch);
                const coordPos = fromCanvasCoords(touchPos);
                
                // Check if we touched a vertex
                for (let i = 0; i < vertices.length; i++) {
                    const vertexCanvasPos = toCanvasCoords(vertices[i]);
                    if (distance(touchPos, vertexCanvasPos) < 25) { // Larger touch area
                        selectedVertex = i;
                        break;
                    }
                }
            }
        }
        
        function onTouchMove(e) {
            e.preventDefault();
            if (selectedVertex !== null && e.touches.length === 1) {
                const touch = e.touches[0];
                const touchPos = getTouchPos(triangleCanvas, touch);
                const coordPos = fromCanvasCoords(touchPos);
                
                vertices[selectedVertex].x = coordPos.x;
                vertices[selectedVertex].y = coordPos.y;
                drawTriangle();
            }
        }
        
        function onTouchEnd(e) {
            selectedVertex = null;
        }
        
        // Helper functions for mouse/touch positions
        function getMousePos(canvas, evt) {
            const rect = canvas.getBoundingClientRect();
            return {
                x: evt.clientX - rect.left,
                y: evt.clientY - rect.top
            };
        }
        
        function getTouchPos(canvas, touch) {
            const rect = canvas.getBoundingClientRect();
            return {
                x: touch.clientX - rect.left,
                y: touch.clientY - rect.top
            };
        }
        
        // Start the application
        window.onload = init;
    </script>"""

HTML(html_content)